<a href="https://colab.research.google.com/github/pavstar619/NahidSir_prototype_bdpolice_V2/blob/main/NahidSir_prototype_bdpolice_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install XlsxWriter

     |████████████████████████████████| 153kB 5.3MB/s 


In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPClassifier
from sklearn import linear_model 
from sklearn.linear_model import LinearRegression

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

from numpy import array
from sklearn.model_selection import KFold



In [3]:
pd.set_option('display.max_columns', None) # show all columns when printing


In [4]:
def R2_calc(pred_y, test_y): # Closer to 1 better score
  return r2_score(pred_y, test_y)

def MSE_calc(pred_y, test_y): # Lower the better
  return mean_squared_error(pred_y, test_y, squared = False)
  
def MAE_calc(pred_y, test_y): # Lower the better
  return mean_absolute_error(pred_y, test_y)

def Error_calc(pred_y, test_y):
  return (pred_y - test_y) / test_y

In [5]:
def modelize(dmp,single_data):
  # Create a Pandas Excel writer using XlsxWriter as the engine.
  writer = pd.ExcelWriter('With2018train_test_pred_data.xlsx', engine='xlsxwriter')
  # writer1 = pd.ExcelWriter('With2018train_errors_data.xlsx', engine='xlsxwriter')

  # dmp_x = dmp.loc[:, dmp.columns != 'Robbery']
  # dmp_y = dmp['Robbery']
  for col in dmp.columns[1:].unique():
    print('~~~~~~~~~~~~~~~~~~~~~', col, '~~~~~~~~~~~~~~~~~~~~~')
    dmp_x = dmp['Year'].values
    dmp_y = dmp[col].values

    train_x, test_x, train_y, test_y = train_test_split(dmp_x, dmp_y, test_size = 0.25, random_state=0)

    # Standard scaling
    ss = StandardScaler()
    ss.fit(train_x.reshape(-1, 1))
    ss_train_x = ss.transform(train_x.reshape(-1, 1))
    ss_test_x = ss.transform(test_x.reshape(-1, 1))

    print(train_x)
    print(test_x)




    # pred for single data
    Xnew = [[single_data]]
    # print(len(Xnew[0]))
    # print(len(dmp_x.columns))




    regressor = DecisionTreeRegressor(random_state=0)
    forest = RandomForestRegressor(max_depth=2, random_state=0)
    svr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
    mlp = MLPClassifier(random_state=0, max_iter = 2000)
    lasso = linear_model.Lasso(alpha=0.1)
    bayesian = linear_model.BayesianRidge()
    ridge = linear_model.Ridge()
    lr = LinearRegression()

    regressor.fit(train_x.reshape(-1, 1),train_y)  
    forest.fit(train_x.reshape(-1, 1),train_y) 
    svr.fit(train_x.reshape(-1, 1),train_y)
    mlp.fit(ss_train_x.reshape(-1,1),train_y) 
    lasso.fit(train_x.reshape(-1,1),train_y) 
    bayesian.fit(train_x.reshape(-1,1),train_y) 
    ridge.fit(train_x.reshape(-1,1),train_y) 
    lr.fit(train_x.reshape(-1,1),train_y)


    pred_y = regressor.predict(test_x.reshape(-1, 1))
    pred_y1 = forest.predict(test_x.reshape(-1, 1)) 
    pred_y2 = svr.predict(test_x.reshape(-1, 1)) 
    pred_y3 = mlp.predict(ss_test_x.reshape(-1, 1))
    pred_y4 = lasso.predict(test_x.reshape(-1,1))
    pred_y5 = bayesian.predict(test_x.reshape(-1,1))
    pred_y6 = ridge.predict(test_x.reshape(-1,1))
    pred_y7 = lr.predict(test_x.reshape(-1,1))

    # error_y = (pred_y - test_y) / test_y #lower the better
    # error_y1 = (pred_y1 - test_y) / test_y
    # error_y2 = (pred_y2 - test_y) / test_y
    # error_y3 = (pred_y3 - test_y) / test_y
    # error_y4 = (pred_y4 - test_y) / test_y
    # error_y5 = (pred_y5 - test_y) / test_y
    # error_y6 = (pred_y6 - test_y) / test_y
    # error_y7 = (pred_y7 - test_y) / test_y


    pred_y_list = [pred_y,pred_y1,pred_y2,pred_y3,pred_y4,pred_y5,pred_y6,pred_y7]
    r2_y_list = []
    mse_y_list = []
    mae_y_list = []

    for pred_y in pred_y_list:
      r2_y = R2_calc(pred_y,test_y)
      r2_y_list.append(r2_y)

      mse_y = MSE_calc(pred_y,test_y)
      mse_y_list.append(mse_y)

      mae_y = MAE_calc(pred_y,test_y)
      mae_y_list.append(mae_y)

    # print('R2 SCORES: ', r2_y_list)
    # print('RMSE SCORES: ', mse_y_list)
    # print('MAE SCORES: ', mae_y_list)

    Xpred_y = mlp.predict(Xnew) #one data only

    model_list = ['Decision Tree', 'Random Forest', 'SVR', 'MLP(Adam)', 'Lasso', 'Bayesian', 'Ridge', 'Linear Regression']
    # print('~~~~ Test Val || Predicted Val || Errors: R2, RMSE, MAE ~~~~')
    for i in range(len(pred_y_list)):
      print(model_list[i], 'Test Val: ',test_y, 'Predicted Val: ', pred_y_list[i], 'Errors R2 Score | RMSE | MAE: ', r2_y_list[i], mse_y_list[i], mae_y_list[i])


    # # Put in dataframes
    # errors_data = pd.DataFrame(list(zip(model_list, r2_y_list, mse_y_list, mae_y_list)),
    #            columns =['Algorithms','R2 Score','Mean Square Error(MSE)','Mean Absolute Error(MAE)'])
    # # print(errors_data)
    test_pred_data = pd.DataFrame(columns =['Algorithms','Test Years','Test Values','Predicted Values','R2 Score','Mean Square Error(MSE)','Mean Absolute Error(MAE)'])
    for i in range(len(pred_y_list)):
      test_pred_data.loc[i] = [model_list[i], test_x, test_y, list(np.around(pred_y_list[i],2)), r2_y_list[i], mse_y_list[i], mae_y_list[i]]
    # print(test_pred_data)


    # Put in excel sheets
    # Write each dataframe to a different worksheet.
    test_pred_data.to_excel(writer, sheet_name=col,index=False)
    # errors_data.to_excel(writer1, sheet_name=col,index=False)


    # print(test_y)
    # print('Test DT: ',test_y,pred_y) # Decision Tree
    # print('Test RF: ',test_y,pred_y1) # Random Forest
    # print('Test SVR: ',test_y,pred_y2) # SVM
    # print('Test MLP: ',test_y,pred_y3) # mlp
    # print('Test Lasso: ',test_y,pred_y4) # lasso
    # print('Test Bayesian: ',test_y,pred_y5) # bayesian
    # print('Test Ridge: ',test_y,pred_y6) # ridge
    # print('Test Linear Regression: ',test_y,pred_y7) # lr

    print(Xnew,'BEST MLP: ',Xpred_y) # Decision Tree

  # Close the Pandas Excel writer and output the Excel file.
  writer.save()
  # writer1.save()



def modelizeall(dmp, dmp_2018):
  # Create a Pandas Excel writer using XlsxWriter as the engine.
  writer = pd.ExcelWriter('Without2018train_test_pred_data.xlsx', engine='xlsxwriter')
  # writer1 = pd.ExcelWriter('Without2018train_errors_data.xlsx', engine='xlsxwriter')

  # dmp_x = dmp.loc[:, dmp.columns != 'Robbery']
  # dmp_y = dmp['Robbery']
  for col in dmp.columns[1:].unique():
    print('~~~~~~~~~~~~~~~~~~~~~', col, '~~~~~~~~~~~~~~~~~~~~~')
    dmp_x = dmp['Year'].values
    dmp_y = dmp[col].values

    # ASSIGN MANUALLY
    train_x = dmp_x
    train_y = dmp_y

    test_x = dmp_2018['Year'].values
    test_y = dmp_2018[col].values


    # train_x, test_x, train_y, test_y = train_test_split(dmp_x, dmp_y, test_size = 0.25, random_state=0)

    # Standard scaling
    ss = StandardScaler()
    ss.fit(train_x.reshape(-1, 1))
    ss_train_x = ss.transform(train_x.reshape(-1, 1))
    ss_test_x = ss.transform(test_x.reshape(-1, 1))

    print(train_x)
    # print(test_x)



    # pred for single data
    # Xnew = [[single_data]]
    # print(len(Xnew[0]))
    # print(len(dmp_x.columns))

    regressor = DecisionTreeRegressor(random_state=0)
    forest = RandomForestRegressor(max_depth=2, random_state=0)
    svr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
    mlp = MLPClassifier(random_state=0, max_iter = 2000)
    lasso = linear_model.Lasso(alpha=0.1)
    bayesian = linear_model.BayesianRidge()
    ridge = linear_model.Ridge()
    lr = LinearRegression()

    regressor.fit(train_x.reshape(-1, 1),train_y)  
    forest.fit(train_x.reshape(-1, 1),train_y) 
    svr.fit(train_x.reshape(-1, 1),train_y)
    mlp.fit(ss_train_x.reshape(-1,1),train_y) 
    lasso.fit(train_x.reshape(-1,1),train_y) 
    bayesian.fit(train_x.reshape(-1,1),train_y) 
    ridge.fit(train_x.reshape(-1,1),train_y) 
    lr.fit(train_x.reshape(-1,1),train_y)


    pred_y = regressor.predict(test_x.reshape(-1, 1))
    pred_y1 = forest.predict(test_x.reshape(-1, 1)) 
    pred_y2 = svr.predict(test_x.reshape(-1, 1)) 
    pred_y3 = mlp.predict(ss_test_x.reshape(-1, 1))
    pred_y4 = lasso.predict(test_x.reshape(-1,1))
    pred_y5 = bayesian.predict(test_x.reshape(-1,1))
    pred_y6 = ridge.predict(test_x.reshape(-1,1))
    pred_y7 = lr.predict(test_x.reshape(-1,1))

    # error_y = (pred_y - test_y) / test_y #lower the better
    # error_y1 = (pred_y1 - test_y) / test_y
    # error_y2 = (pred_y2 - test_y) / test_y
    # error_y3 = (pred_y3 - test_y) / test_y
    # error_y4 = (pred_y4 - test_y) / test_y
    # error_y5 = (pred_y5 - test_y) / test_y
    # error_y6 = (pred_y6 - test_y) / test_y
    # error_y7 = (pred_y7 - test_y) / test_y

    pred_y_list = [pred_y,pred_y1,pred_y2,pred_y3,pred_y4,pred_y5,pred_y6,pred_y7]
    error_y_list = []
    r2_y_list = []
    mse_y_list = []
    mae_y_list = []

    for pred_y in pred_y_list:
      # error_y = Error_calc(pred_y,test_y)
      # error_y_list.append(error_y)

      r2_y = R2_calc(pred_y,test_y)
      r2_y_list.append(r2_y)

      mse_y = MSE_calc(pred_y,test_y)
      mse_y_list.append(mse_y)

      mae_y = MAE_calc(pred_y,test_y)
      mae_y_list.append(mae_y)


    model_list = ['Decision Tree', 'Random Forest', 'SVR', 'MLP(Adam)', 'Lasso', 'Bayesian', 'Ridge', 'Linear Regression']
    # print('~~~~ Test Val || Predicted Val || Errors: R2, RMSE, MAE ~~~~')
    for i in range(len(pred_y_list)):
      print(model_list[i], 'Test Val: ',test_y, 'Predicted Val: ', pred_y_list[i], 'Errors R2 | RMSE | MAE: ', r2_y_list[i], mse_y_list[i], mae_y_list[i]) # Dont show Metric as only one test row


    # # Put in dataframes
    # errors_data = pd.DataFrame(list(zip(model_list, r2_y_list, mse_y_list, mae_y_list)),
    #            columns =['Algorithms','R2 Score','Mean Square Error(MSE)','Mean Absolute Error(MAE)'])
    # # print(errors_data)
    test_pred_data = pd.DataFrame(columns =['Algorithms','Test Years','Test Values','Predicted Values','R2 Score','Mean Square Error(MSE)','Mean Absolute Error(MAE)'])
    for i in range(len(pred_y_list)):
      test_pred_data.loc[i] = [model_list[i], test_x, test_y, list(np.around(pred_y_list[i],2)), r2_y_list[i], mse_y_list[i], mae_y_list[i]]
    # print(test_pred_data)


    # Put in excel sheets
    # Write each dataframe to a different worksheet.
    test_pred_data.to_excel(writer, sheet_name=col,index=False)
    # errors_data.to_excel(writer1, sheet_name=col,index=False)


    # Xpred_y = mlp.predict(Xnew) #one data only


    # m2_y = mean_squared_error(test_y,pred_y, squared=False)

    # print('~~~~ Test Val || Predicted Val || Errors: R2, RMSE, RSE ~~~~')
    # # print(test_y)
    # print('Test DT: ',test_y,pred_y,error_y) # Decision Tree
    # print('Test RF: ',test_y,pred_y1,error_y1) # Random Forest
    # print('Test SVR: ',test_y,pred_y2,error_y2) # SVM
    # print('Test MLP: ',test_y,pred_y3,error_y3) # mlp
    # print('Test Lasso: ',test_y,pred_y4,error_y4) # lasso
    # print('Test Bayesian: ',test_y,pred_y5,error_y5) # bayesian
    # print('Test Ridge: ',test_y,pred_y6,error_y6) # ridge
    # print('Test Linear Regression: ',test_y,pred_y7,error_y7) # lr

    print('2018 BEST MLP: ',test_y,pred_y3) # Decision Tree

  # Close the Pandas Excel writer and output the Excel file.
  writer.save()
  # writer1.save()


Model With 2019


In [6]:
#@title Code here
# # dmp = pd.read_csv('dmp.csv') # for dhaka only
# dmp = pd.read_excel('bdpolice_totals.xlsx')  
# print(dmp)

# # dmp = dmp.drop(9) # renove 2019 as data not accurate
# dmp

In [7]:
#@title Code here
# # dmp_x = dmp.loc[:, dmp.columns != 'Robbery']
# # dmp_y = dmp['Robbery']
# for col in dmp.columns[1:].unique():
#   print('~~~~~~~~~~~~~~~~~~~~~', col, '~~~~~~~~~~~~~~~~~~~~~')
#   dmp_x = dmp['Year'].values
#   dmp_y = dmp[col].values

#   train_x, test_x, train_y, test_y = train_test_split(dmp_x, dmp_y, test_size = 0.25, random_state=1)

#   # Standard scaling
#   ss = StandardScaler()
#   ss.fit(train_x.reshape(-1, 1))
#   ss_train_x = ss.transform(train_x.reshape(-1, 1))
#   ss_test_x = ss.transform(test_x.reshape(-1, 1))

#   print(dmp_x)
#   print(dmp_y)



#   # pred for single data
#   Xnew = [[2020]]
#   print(len(Xnew[0]))
#   # print(len(dmp_x.columns))




#   regressor = DecisionTreeRegressor(random_state=0)
#   forest = RandomForestRegressor(max_depth=2, random_state=0)
#   svr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
#   mlp = MLPClassifier(random_state=0, max_iter = 2000)
#   lasso = linear_model.Lasso(alpha=0.1)
#   bayesian = linear_model.BayesianRidge()
#   ridge = linear_model.Ridge()
#   lr = LinearRegression()

#   regressor.fit(train_x.reshape(-1, 1),train_y)  
#   forest.fit(train_x.reshape(-1, 1),train_y) 
#   svr.fit(train_x.reshape(-1, 1),train_y)
#   mlp.fit(ss_train_x.reshape(-1,1),train_y) 
#   lasso.fit(train_x.reshape(-1,1),train_y) 
#   bayesian.fit(train_x.reshape(-1,1),train_y) 
#   ridge.fit(train_x.reshape(-1,1),train_y) 
#   lr.fit(train_x.reshape(-1,1),train_y)


#   pred_y = regressor.predict(test_x.reshape(-1, 1))
#   pred_y1 = forest.predict(test_x.reshape(-1, 1)) 
#   pred_y2 = svr.predict(test_x.reshape(-1, 1)) 
#   pred_y3 = mlp.predict(ss_test_x.reshape(-1, 1))
#   pred_y4 = lasso.predict(test_x.reshape(-1,1))
#   pred_y5 = bayesian.predict(test_x.reshape(-1,1))
#   pred_y6 = ridge.predict(test_x.reshape(-1,1))
#   pred_y7 = ridge.predict(test_x.reshape(-1,1))

#   error_y = (pred_y - test_y) / test_y #lower the better
#   error_y1 = (pred_y1 - test_y) / test_y
#   error_y2 = (pred_y2 - test_y) / test_y
#   error_y3 = (pred_y3 - test_y) / test_y
#   error_y4 = (pred_y4 - test_y) / test_y
#   error_y5 = (pred_y5 - test_y) / test_y
#   error_y6 = (pred_y6 - test_y) / test_y
#   error_y7 = (pred_y7 - test_y) / test_y


#   Xpred_y = mlp.predict(Xnew) #one data only

#   # m2_y = mean_squared_error(test_y,pred_y, squared=False)

#   # print(test_y)
#   print('Test DT: ',test_y,pred_y,error_y) # Decision Tree
#   print('Test RF: ',test_y,pred_y1,error_y1) # Random Forest
#   print('Test SVR: ',test_y,pred_y2,error_y2) # SVM
#   print('Test MLP: ',test_y,pred_y3,error_y3) # mlp
#   print('Test Lasso: ',test_y,pred_y4,error_y4) # lasso
#   print('Test Bayesian: ',test_y,pred_y5,error_y5) # bayesian
#   print('Test Ridge: ',test_y,pred_y6,error_y6) # ridge
#   print('Test Linear Regression: ',test_y,pred_y7,error_y7) # lr

#   print('2019 BEST MLP: ',Xnew,Xpred_y) # Decision Tree


Model Without 2019

In [8]:
# dmp = pd.read_csv('dmp.csv')
dmp = pd.read_excel('bdpolice_totals.xlsx')  
# print(dmp)

dmp = dmp.drop(9) # renove 2019 as data not accurate
# dmp

Model 2010 to 2018 (With Train Test Split)


In [9]:
print(dmp)

   Year  Dacoity  Robbery  Murder  Speedy Trial  Riot  \
0  2010      656     1059    3988          1666   130   
1  2011      650     1069    3966          1863   109   
2  2012      593      964    4114          1907    94   
3  2013      613     1021    4393          1896   172   
4  2014      651     1155    4514          1716    79   
5  2015      492      933    4037          1549    93   
6  2016      408      722    3591          1052    53   
7  2017      336      657    3549          1045    23   
8  2018      262      562    3830           922    26   

   Woman & Child Repression  Kidnapping  Police Assault  Burglary  Theft  \
0                     17752         870             473      3101   8529   
1                     21389         792             581      3134   8873   
2                     20947         850             659      2927   8598   
3                     19601         879            1257      2762   7882   
4                     21291         920          

In [10]:
single_data = 2019
modelize(dmp,single_data)

~~~~~~~~~~~~~~~~~~~~~ Dacoity ~~~~~~~~~~~~~~~~~~~~~
[2014 2018 2016 2013 2010 2015]
[2017 2012 2011]
Decision Tree Test Val:  [336 593 650] Predicted Val:  [408. 613. 656.] Errors R2 Score | RMSE | MAE:  0.8400045550304618 43.282020901678486 32.666666666666664
Random Forest Test Val:  [336 593 650] Predicted Val:  [393.75       630.09583333 638.24416667] Errors R2 Score | RMSE | MAE:  0.8742636701616564 40.20515293697048 35.53388888888886
SVR Test Val:  [336 593 650] Predicted Val:  [550.27529438 553.78014155 553.70987211] Errors R2 Score | RMSE | MAE:  -7064.399169538658 137.5062275013877 116.59509357028942
MLP(Adam) Test Val:  [336 593 650] Predicted Val:  [262 613 656] Errors R2 Score | RMSE | MAE:  0.9367217770420213 44.39219150556398 33.333333333333336
Lasso Test Val:  [336 593 650] Predicted Val:  [377.4 632.9 684. ] Errors R2 Score | RMSE | MAE:  0.9173173455242492 38.56583116352877 38.43333333333916
Bayesian Test Val:  [336 593 650] Predicted Val:  [384.73835002 626.47894373 67

Model 2010 to 2017 (Entire data train)


In [11]:
# 2018 dmp
dmp_2018 = dmp.iloc[8:]
print(dmp_2018)

# 2010 to 2017
dmp_till2017 = dmp.drop(8)
print(dmp_till2017)



   Year  Dacoity  Robbery  Murder  Speedy Trial  Riot  \
8  2018      262      562    3830           922    26   

   Woman & Child Repression  Kidnapping  Police Assault  Burglary  Theft  \
8                     16253         444             811      2137   5561   

   Arms Act  Explosive  Narcotics  Smuggling  Other Cases  
8      2515       1310     112549       4501        69736  
   Year  Dacoity  Robbery  Murder  Speedy Trial  Riot  \
0  2010      656     1059    3988          1666   130   
1  2011      650     1069    3966          1863   109   
2  2012      593      964    4114          1907    94   
3  2013      613     1021    4393          1896   172   
4  2014      651     1155    4514          1716    79   
5  2015      492      933    4037          1549    93   
6  2016      408      722    3591          1052    53   
7  2017      336      657    3549          1045    23   

   Woman & Child Repression  Kidnapping  Police Assault  Burglary  Theft  \
0                     

In [12]:
modelizeall(dmp_till2017, dmp_2018)

~~~~~~~~~~~~~~~~~~~~~ Dacoity ~~~~~~~~~~~~~~~~~~~~~
[2010 2011 2012 2013 2014 2015 2016 2017]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetri

Decision Tree Test Val:  [262] Predicted Val:  [336.] Errors R2 | RMSE | MAE:  nan 74.0 74.0
Random Forest Test Val:  [262] Predicted Val:  [374.34] Errors R2 | RMSE | MAE:  nan 112.33999999999997 112.33999999999997
SVR Test Val:  [262] Predicted Val:  [600.43007918] Errors R2 | RMSE | MAE:  nan 338.4300791766442 338.4300791766442
MLP(Adam) Test Val:  [262] Predicted Val:  [336] Errors R2 | RMSE | MAE:  nan 74.0 74.0
Lasso Test Val:  [262] Predicted Val:  [350.94285714] Errors R2 | RMSE | MAE:  nan 88.94285714285797 88.94285714285797
Bayesian Test Val:  [262] Predicted Val:  [359.31163892] Errors R2 | RMSE | MAE:  nan 97.31163892017503 97.31163892017503
Ridge Test Val:  [262] Predicted Val:  [355.48546512] Errors R2 | RMSE | MAE:  nan 93.48546511627501 93.48546511627501
Linear Regression Test Val:  [262] Predicted Val:  [350.85714286] Errors R2 | RMSE | MAE:  nan 88.85714285714494 88.85714285714494
2018 BEST MLP:  [262] [336]
~~~~~~~~~~~~~~~~~~~~~ Robbery ~~~~~~~~~~~~~~~~~~~~~
[2010 20

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetri

Decision Tree Test Val:  [562] Predicted Val:  [657.] Errors R2 | RMSE | MAE:  nan 95.0 95.0
Random Forest Test Val:  [562] Predicted Val:  [699.12] Errors R2 | RMSE | MAE:  nan 137.12 137.12
SVR Test Val:  [562] Predicted Val:  [989.93007918] Errors R2 | RMSE | MAE:  nan 427.9300791766442 427.9300791766442
MLP(Adam) Test Val:  [562] Predicted Val:  [657] Errors R2 | RMSE | MAE:  nan 95.0 95.0
Lasso Test Val:  [562] Predicted Val:  [706.08571429] Errors R2 | RMSE | MAE:  nan 144.08571428571304 144.08571428571304
Bayesian Test Val:  [562] Predicted Val:  [731.78603839] Errors R2 | RMSE | MAE:  nan 169.78603839498828 169.78603839498828
Ridge Test Val:  [562] Predicted Val:  [711.61627907] Errors R2 | RMSE | MAE:  nan 149.61627906977083 149.61627906977083
Linear Regression Test Val:  [562] Predicted Val:  [706.] Errors R2 | RMSE | MAE:  nan 144.0 144.0
2018 BEST MLP:  [562] [657]
~~~~~~~~~~~~~~~~~~~~~ Murder ~~~~~~~~~~~~~~~~~~~~~
[2010 2011 2012 2013 2014 2015 2016 2017]


...

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetri

Decision Tree Test Val:  [112549] Predicted Val:  [98984.] Errors R2 | RMSE | MAE:  nan 13565.0 13565.0
Random Forest Test Val:  [112549] Predicted Val:  [85923.195] Errors R2 | RMSE | MAE:  nan 26625.804999999993 26625.804999999993
SVR Test Val:  [112549] Predicted Val:  [39884.74345596] Errors R2 | RMSE | MAE:  nan 72664.25654404334 72664.25654404334
MLP(Adam) Test Val:  [112549] Predicted Val:  [98984] Errors R2 | RMSE | MAE:  nan 13565.0 13565.0
Lasso Test Val:  [112549] Predicted Val:  [84503.66428571] Errors R2 | RMSE | MAE:  nan 28045.335714286193 28045.335714286193
Bayesian Test Val:  [112549] Predicted Val:  [48186.87934476] Errors R2 | RMSE | MAE:  nan 64362.12065524275 64362.12065524275
Ridge Test Val:  [112549] Predicted Val:  [83659.17151163] Errors R2 | RMSE | MAE:  nan 28889.828488372266 28889.828488372266
Linear Regression Test Val:  [112549] Predicted Val:  [84503.75] Errors R2 | RMSE | MAE:  nan 28045.25 28045.25
2018 BEST MLP:  [112549] [98984]
~~~~~~~~~~~~~~~~~~~~~ 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetri

Decision Tree Test Val:  [4501] Predicted Val:  [5599.] Errors R2 | RMSE | MAE:  nan 1098.0 1098.0
Random Forest Test Val:  [4501] Predicted Val:  [5439.92] Errors R2 | RMSE | MAE:  nan 938.9200000000001 938.9200000000001
SVR Test Val:  [4501] Predicted Val:  [6269.58392781] Errors R2 | RMSE | MAE:  nan 1768.5839278060894 1768.5839278060894
MLP(Adam) Test Val:  [4501] Predicted Val:  [5599] Errors R2 | RMSE | MAE:  nan 1098.0 1098.0
Lasso Test Val:  [4501] Predicted Val:  [5433.55] Errors R2 | RMSE | MAE:  nan 932.5499999999884 932.5499999999884
Bayesian Test Val:  [4501] Predicted Val:  [5717.653916] Errors R2 | RMSE | MAE:  nan 1216.6539159981185 1216.6539159981185
Ridge Test Val:  [4501] Predicted Val:  [5447.62209302] Errors R2 | RMSE | MAE:  nan 946.6220930232666 946.6220930232666
Linear Regression Test Val:  [4501] Predicted Val:  [5433.46428571] Errors R2 | RMSE | MAE:  nan 932.4642857142608 932.4642857142608
2018 BEST MLP:  [4501] [5599]
~~~~~~~~~~~~~~~~~~~~~ Other Cases ~~~~~~

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetri

In [13]:
#@title Hidden code
# # dmp_x = dmp.loc[:, dmp.columns != 'Robbery']
# # dmp_y = dmp['Robbery']
# for col in dmp.columns[1:].unique():
#   print('~~~~~~~~~~~~~~~~~~~~~', col, '~~~~~~~~~~~~~~~~~~~~~')
#   dmp_x = dmp['Year'].values
#   dmp_y = dmp[col].values

#   train_x, test_x, train_y, test_y = train_test_split(dmp_x, dmp_y, test_size = 0.25, random_state=0)

#   # Standard scaling
#   ss = StandardScaler()
#   ss.fit(train_x.reshape(-1, 1))
#   ss_train_x = ss.transform(train_x.reshape(-1, 1))
#   ss_test_x = ss.transform(test_x.reshape(-1, 1))

#   print(train_x)
#   print(test_x)



#   # pred for single data
#   Xnew = [[2018]]
#   print(len(Xnew[0]))
#   # print(len(dmp_x.columns))




#   regressor = DecisionTreeRegressor(random_state=0)
#   forest = RandomForestRegressor(max_depth=2, random_state=0)
#   svr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
#   mlp = MLPClassifier(random_state=0, max_iter = 2000)
#   lasso = linear_model.Lasso(alpha=0.1)
#   bayesian = linear_model.BayesianRidge()
#   ridge = linear_model.Ridge()
#   lr = LinearRegression()

#   regressor.fit(train_x.reshape(-1, 1),train_y)  
#   forest.fit(train_x.reshape(-1, 1),train_y) 
#   svr.fit(train_x.reshape(-1, 1),train_y)
#   mlp.fit(ss_train_x.reshape(-1,1),train_y) 
#   lasso.fit(train_x.reshape(-1,1),train_y) 
#   bayesian.fit(train_x.reshape(-1,1),train_y) 
#   ridge.fit(train_x.reshape(-1,1),train_y) 
#   lr.fit(train_x.reshape(-1,1),train_y)


#   pred_y = regressor.predict(test_x.reshape(-1, 1))
#   pred_y1 = forest.predict(test_x.reshape(-1, 1)) 
#   pred_y2 = svr.predict(test_x.reshape(-1, 1)) 
#   pred_y3 = mlp.predict(ss_test_x.reshape(-1, 1))
#   pred_y4 = lasso.predict(test_x.reshape(-1,1))
#   pred_y5 = bayesian.predict(test_x.reshape(-1,1))
#   pred_y6 = ridge.predict(test_x.reshape(-1,1))
#   pred_y7 = lr.predict(test_x.reshape(-1,1))

#   error_y = (pred_y - test_y) / test_y #lower the better
#   error_y1 = (pred_y1 - test_y) / test_y
#   error_y2 = (pred_y2 - test_y) / test_y
#   error_y3 = (pred_y3 - test_y) / test_y
#   error_y4 = (pred_y4 - test_y) / test_y
#   error_y5 = (pred_y5 - test_y) / test_y
#   error_y6 = (pred_y6 - test_y) / test_y
#   error_y7 = (pred_y7 - test_y) / test_y


#   Xpred_y = mlp.predict(Xnew) #one data only


#   # m2_y = mean_squared_error(test_y,pred_y, squared=False)

#   # print(test_y)
#   print('Test DT: ',test_y,pred_y,error_y) # Decision Tree
#   print('Test RF: ',test_y,pred_y1,error_y1) # Random Forest
#   print('Test SVR: ',test_y,pred_y2,error_y2) # SVM
#   print('Test MLP: ',test_y,pred_y3,error_y3) # mlp
#   print('Test Lasso: ',test_y,pred_y4,error_y4) # lasso
#   print('Test Bayesian: ',test_y,pred_y5,error_y5) # bayesian
#   print('Test Ridge: ',test_y,pred_y6,error_y6) # ridge
#   print('Test Linear Regression: ',test_y,pred_y7,error_y7) # lr

#   print('2018 BEST MLP: ',Xnew,Xpred_y) # Decision Tree




K Fold Stuff

In [14]:
# # scikit-learn k-fold cross-validation
# from numpy import array
# from sklearn.model_selection import KFold
# # data sample
# data = array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6])
# # prepare cross validation
# kfold = KFold(3, True, 1)
# # enumerate splits
# for train, test in kfold.split(data):
# 	print('train: %s, test: %s' % (data[train], data[test]))

In [15]:
def kfoldize(dmp, dmp_2018, kfoldnumber):
  kfold = KFold(kfoldnumber, True, 1)
  writer = pd.ExcelWriter('Kfoldtrain_test_pred_errors_data.xlsx', engine='xlsxwriter')

  for col in dmp.columns[1:].unique():
    print('~~~~~~~~~~~~~~~~~~~~~', col, '~~~~~~~~~~~~~~~~~~~~~')
    dmp_x = dmp['Year'].values
    dmp_y = dmp[col].values

    dmp_2018_x = dmp_2018['Year'].values
    dmp_2018_y = dmp_2018[col].values

    count = 0
    test_pred_error_data = pd.DataFrame(columns =['Kfold Iteration','Algorithms','Test Years','Test Values','Predicted Values','R2 Score','Mean Square Error(MSE)','Mean Absolute Error(MAE)'])

    for train_index, test_index in kfold.split(dmp_x):
      # all kfolds train test splits
      print('train: %s, test: %s' % (dmp_x[train_index], dmp_x[test_index]))
      print('train: %s, test: %s' % (dmp_y[train_index], dmp_y[test_index]))
      # print("TRAIN:", train_index, "TEST:", test_index)

      # mlp = MLPClassifier(random_state=0, max_iter = 2000) 
      mlp = make_pipeline(StandardScaler(), MLPClassifier(random_state=0, max_iter = 2000))
      # mlp = LinearRegression() # Test purposes
      mlp.fit(dmp_x[train_index].reshape(-1,1),dmp_y[train_index])
      pred_y3 = mlp.predict(dmp_x[test_index].reshape(-1, 1))

      # predict 2018
      Xpred_y = mlp.predict(dmp_2018_x.reshape(-1, 1))

      r2_y = R2_calc(pred_y3, dmp_y[test_index])
      mse_y = MSE_calc(pred_y3, dmp_y[test_index])
      mae_y = MAE_calc(pred_y3, dmp_y[test_index])


      # Put in dataframes
      # errors_data = pd.DataFrame('MLP',r2_y,mse_y,mae_y,
      #           columns =['Algorithms','R2 Score','Mean Square Error(MSE)','Mean Absolute Error(MAE)'])
      # print(errors_data)
      test_pred_error_data.loc[count] = [count,'MLP', dmp_x[test_index],dmp_y[test_index], np.around(pred_y3,2), r2_y, mse_y, mae_y]
      count+=1
      print(count)
      



      print('Test Val: ',dmp_y[test_index], 'Predicted Val: ', pred_y3, 'R2 | RMSE | MAE Errors', r2_y, mse_y, mae_y)

      # print(precision_recall_fscore_support(dmp_y[test_index], pred_y3, average='macro'))
      # error_y3 = (pred_y3 - dmp_y[test_index]) / dmp_y[test_index]
      # print(error_y3)

      print(dmp_2018_x[0], 'BEST MLP: ', 'Test Val: ', dmp_2018_y[0], 'Predicted Val: ',Xpred_y) # Decision Tree

      print('\n')

    # print(test_pred_error_data)
    test_pred_error_data.to_excel(writer, sheet_name=col,index=False)

  writer.save()



In [16]:
kfoldize(dmp_till2017, dmp_2018, 5) # kfold with 5 folds

~~~~~~~~~~~~~~~~~~~~~ Dacoity ~~~~~~~~~~~~~~~~~~~~~
train: [2010 2011 2013 2014 2015 2016], test: [2012 2017]
train: [656 650 613 651 492 408], test: [593 336]
1
Test Val:  [593 336] Predicted Val:  [613 408] R2 | RMSE | MAE Errors 0.7342534205829863 52.839379254491625 46.0
2018 BEST MLP:  Test Val:  262 Predicted Val:  [408]


train: [2010 2012 2013 2014 2015 2017], test: [2011 2016]
train: [656 593 613 651 492 336], test: [650 408]
2
Test Val:  [650 408] Predicted Val:  [656 336] R2 | RMSE | MAE Errors 0.898046875 51.088159097779204 39.0
2018 BEST MLP:  Test Val:  262 Predicted Val:  [336]


train: [2011 2012 2013 2015 2016 2017], test: [2010 2014]
train: [650 593 613 492 408 336], test: [656 651]
3
Test Val:  [656 651] Predicted Val:  [650 613] R2 | RMSE | MAE Errors -1.1621621621621623 27.202941017470888 22.0
2018 BEST MLP:  Test Val:  262 Predicted Val:  [336]


train: [2010 2011 2012 2014 2015 2016 2017], test: [2013]
train: [656 650 593 651 492 408 336], test: [613]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


4
Test Val:  [613] Predicted Val:  [651] R2 | RMSE | MAE Errors nan 38.0 38.0
2018 BEST MLP:  Test Val:  262 Predicted Val:  [336]


train: [2010 2011 2012 2013 2014 2016 2017], test: [2015]
train: [656 650 593 613 651 408 336], test: [492]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


5
Test Val:  [492] Predicted Val:  [408] R2 | RMSE | MAE Errors nan 84.0 84.0
2018 BEST MLP:  Test Val:  262 Predicted Val:  [336]


~~~~~~~~~~~~~~~~~~~~~ Robbery ~~~~~~~~~~~~~~~~~~~~~
train: [2010 2011 2013 2014 2015 2016], test: [2012 2017]
train: [1059 1069 1021 1155  933  722], test: [964 657]
1
Test Val:  [964 657] Predicted Val:  [1069  722] R2 | RMSE | MAE Errors 0.7466966754976787 87.32124598286491 85.0
2018 BEST MLP:  Test Val:  562 Predicted Val:  [722]


train: [2010 2012 2013 2014 2015 2017], test: [2011 2016]
train: [1059  964 1021 1155  933  657], test: [1069  722]
2
Test Val:  [1069  722] Predicted Val:  [1059  657] R2 | RMSE | MAE Errors 0.9464740971758124 46.502688094345686 37.5
2018 BEST MLP:  Test Val:  562 Predicted Val:  [657]


train: [2011 2012 2013 2015 2016 2017], test: [2010 2014]
train: [1069  964 1021  933  722  657], test: [1059 1155]
3
Test Val:  [1059 1155] Predicted Val:  [1069 1021] R2 | RMSE | MAE Errors -14.67361111111111 95.01578816175763 72.0
2018 B

...

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


5
Test Val:  [6179] Predicted Val:  [6788] R2 | RMSE | MAE Errors nan 609.0 609.0
2018 BEST MLP:  Test Val:  4501 Predicted Val:  [5599]


~~~~~~~~~~~~~~~~~~~~~ Other Cases ~~~~~~~~~~~~~~~~~~~~~
train: [2010 2011 2013 2014 2015 2016], test: [2012 2017]
train: [87139 88355 93930 90400 84117 77747], test: [96112 74645]
1
Test Val:  [96112 74645] Predicted Val:  [93930 77747] R2 | RMSE | MAE Errors 0.890155744280367 2681.746445881862 2642.0
2018 BEST MLP:  Test Val:  69736 Predicted Val:  [77747]


train: [2010 2012 2013 2014 2015 2017], test: [2011 2016]
train: [87139 96112 93930 90400 84117 74645], test: [88355 77747]
2
Test Val:  [88355 77747] Predicted Val:  [87139 74645] R2 | RMSE | MAE Errors 0.8577699238967504 2355.9562814279893 2159.0
2018 BEST MLP:  Test Val:  69736 Predicted Val:  [74645]


train: [2011 2012 2013 2015 2016 2017], test: [2010 2014]
train: [88355 96112 93930 84117 77747 74645], test: [87139 90400]
3
Test Val:  [87139 90400] Predicted Val:  [88355 93930] R2 | RMSE 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


4
Test Val:  [93930] Predicted Val:  [90400] R2 | RMSE | MAE Errors nan 3530.0 3530.0
2018 BEST MLP:  Test Val:  69736 Predicted Val:  [74645]


train: [2010 2011 2012 2013 2014 2016 2017], test: [2015]
train: [87139 88355 96112 93930 90400 77747 74645], test: [84117]
5
Test Val:  [84117] Predicted Val:  [77747] R2 | RMSE | MAE Errors nan 6370.0 6370.0
2018 BEST MLP:  Test Val:  69736 Predicted Val:  [74645]




/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
